# Metaprompt
Welcome to the Metaprompt! This is a prompt engineering tool designed to solve the "blank page problem" and give you a starting point for iteration. All you need to do is enter your task, and optionally the names of the variables you'd like Claude to use in the template. Then you'll be able to run the prompt that comes out on any examples you like.

**Caveats**
- This is designed for single-turn question/response prompts, not multiturn.
- The Metaprompt is designed for use with **Claude 3 Opus.** Generating prompts with other models may lead to worse results.
- The prompt you'll get at the end is not guaranteed to be optimal by any means, so don't be afraid to change it!

### Using This Notebook
The notebook is designed to be maximally easy to use. You don't have to write any code. Just follow these steps:
- Enter your Anthropic API key in between quotation marks where it says "Put your API key here!"
- Enter your task where it says "Replace with your task!"
- Optionally, enter an all-caps list of variables in quotes separated by commas where it says "specify the input variables you want Claude to use".

Then, you can simply click "Runtime -> Run all" and your prompt will be displayed at the bottom of the notebook.

In [ ]:
%pip install anthropic --quiet

# set path to import the meta module that contains the metaprompt
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

import boto3
session = boto3.Session() # create a boto3 session to dynamically get and set the region name
AWS_REGION = session.region_name
print("AWS Region:", AWS_REGION)

# Import python's built-in regular expression library
import re

from anthropic import AnthropicBedrock
CLIENT = AnthropicBedrock(aws_region=AWS_REGION)
MODEL_NAME='anthropic.claude-3-sonnet-20240229-v1:0'

%store MODEL_NAME
%store AWS_REGION

# Table of Contents

0. [The Metaprompt](#0-the-metaprompt)
1. [Quickstart](#1-quickstart) - Enter a task, get a prompt template
2. [Testing your prompt template](#2-testing-your-prompt-template)

## 0. The Metaprompt

The Metaprompt is a long multi-shot prompt filled with half a dozen examples of good prompts for solving various tasks. These examples help Claude to write a good prompt for your task. The full text is below (warning: it's long!)

In [ ]:
# import the meta module from the utils package
# see utils/metap.py for the implementation
# Import the metap module from the utils package
from utils import meta

# 1. Quickstart

Enter your task in the cell below. Here are some examples for inspiration:
- Choose an item from a menu for me given user preferences
- Rate a resume according to a rubric
- Explain a complex scientific concept in simple terms
- Draft an email responding to a customer complaint
- Design a marketing strategy for launching a new product

There are two examples of tasks + optional variables below.

In [ ]:
TASK = "Draft an email responding to a customer complaint" # Replace with your task!
# Optional: specify the input variables you want Claude to use. If you want Claude to choose, you can set `variables` to an empty list!
# VARIABLES = []
VARIABLES = ["CUSTOMER_EMAIL", "COMPANY_NAME"]
# If you want Claude to choose the variables, just leave VARIABLES as an empty list.

# TASK = "Choose an item from a menu for me given my preferences"
# VARIABLES = []
# VARIABLES = ["MENU", "PREFERENCES"]

In [ ]:
variable_string = ""
for variable in VARIABLES:
    variable_string += "\n{" + variable.upper() + "}"
print(variable_string)

Next, we'll insert your task into the metaprompt and see what Claude gives us! Expect this to take 20-30 seconds because the Metaprompt is so long.

In [ ]:
prompt = meta.metaprompt.replace("{{TASK}}", TASK)
assistant_partial = "<Inputs>"
if variable_string:
    assistant_partial += variable_string + "\n</Inputs><Instructions Structure>"

message = CLIENT.messages.create(
    model=MODEL_NAME,
    max_tokens=4096,
    messages=[
        {
            "role": "user",
            "content":  prompt
        },
        {
            "role": "assistant",
            "content": assistant_partial
        }
    ],
    temperature=0
).content[0].text

If you want to see the full text returned by the Metaprompt to see how it planned things out, uncomment out the "pretty_print(message)" line below.

In [ ]:
def pretty_print(message):
    print('\n\n'.join('\n'.join(line.strip() for line in re.findall(r'.{1,100}(?:\s+|$)', paragraph.strip('\n'))) for paragraph in re.split(r'\n\n+', message)))
# pretty_print(message)

Now, we'll extract the prompt itself and the variables needed, while also removing empty tags at the end of the prompt template.

In [ ]:
def extract_between_tags(tag: str, string: str, strip: bool = False) -> list[str]:
    ext_list = re.findall(f"<{tag}>(.+?)</{tag}>", string, re.DOTALL)
    if strip:
        ext_list = [e.strip() for e in ext_list]
    return ext_list

def remove_empty_tags(text):
    return re.sub(r'<(\w+)></\1>$', '', text)

def extract_prompt(metaprompt_response):
    between_tags = extract_between_tags("Instructions", metaprompt_response)[0]
    return remove_empty_tags(remove_empty_tags(between_tags).strip()).strip()

def extract_variables(prompt):
    pattern = r'{([^}]+)}'
    variables = re.findall(pattern, prompt)
    return set(variables)

Below: the variables Claude chose (if you didn't provide any; if you did, these should just be the same ones you provided), and the prompt it wrote.

In [ ]:
extracted_prompt_template = extract_prompt(message)
variables = extract_variables(message)

print("Variables:\n\n" + str(variables))
print("\n************************\n")
print("Prompt:")
pretty_print(extracted_prompt_template)

# 2. Testing your prompt template

If you like your prompt, try it out! The cell will prompt you to add values for each variable. Then, it will be sent to Claude and you'll see Claude's final output.

In [ ]:
variable_values = {}
for variable in variables:
    print("Enter value for variable:", variable)
    variable_values[variable] = input()

prompt_with_variables = extracted_prompt_template
for variable in variable_values:
    prompt_with_variables = prompt_with_variables.replace("{" + variable + "}", variable_values[variable])

message = CLIENT.messages.create(
    model="claude-3-haiku-20240307",
    max_tokens=4096,
    messages=[
        {
            "role": "user",
            "content":  prompt_with_variables
        },
    ],
).content[0].text

print("Claude's output on your prompt:\n\n")
pretty_print(message)